In [1]:
import tensorflow as tf
tf.__version__

'2.2.0'

In [2]:
!nvidia-smi

/bin/sh: nvidia-smi: command not found


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Encoder And Decoder Model

# 0. GPU测试

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# ! unzip drive/'My Drive'/kaikeba/project01/lecture01_code.zip

In [6]:
import warnings
warnings.filterwarnings("ignore")
import sys
sys.path.append('drive/My Drive/kaikeba/project01/code')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from utils.data_loader import build_dataset,load_dataset,preprocess_sentence,load_test_dataset
from utils.wv_loader import load_embedding_matrix,Vocab
from utils.config import *
from gensim.models.word2vec import LineSentence, Word2Vec
from utils.gpu_utils import config_gpu
config_gpu()
import tensorflow as tf
from utils.plot_utils import plot_attention
from tqdm import tqdm
import time
from seq2seq_tf2.seq2seq_batcher import train_batch_generator

Building prefix dict from the default dictionary ...
2020-06-05 09:46:10,927 : DEBUG : Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
2020-06-05 09:46:11,799 : DEBUG : Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.952 seconds.
2020-06-05 09:46:11,885 : DEBUG : Loading model cost 0.952 seconds.
Prefix dict has been built successfully.
2020-06-05 09:46:11,887 : DEBUG : Prefix dict has been built successfully.


1 Physical GPUs, 1 Logical GPUs


# 0. 预处理数据

In [ ]:
# %%time
# build_dataset(train_data_path,test_data_path)

# 1. 加载数据

## 1.1 加载数据集

In [ ]:
params = {}
params["vocab_size"] = 30000
params["embed_size"] = 300
params["enc_units"] = 512
params["attn_units"] = 512
params["dec_units"] = 512
params["batch_size"] = 64
params["epochs"] = 5
params["max_enc_len"] = 200
params["max_dec_len"] = 40

In [13]:
dataset, steps_per_epoch = train_batch_generator(batch_size=params["batch_size"] ,
                                                 max_enc_len=params["max_enc_len"],
                                                 max_dec_len=params["max_dec_len"])
test_X = load_test_dataset(params["max_enc_len"])
steps_per_epoch

1294

In [14]:
test_X.shape

(20000, 200)

## 1.2. 加载vocab

In [15]:
vocab = Vocab(vocab_max_size=params["vocab_size"])

max_size of vocab was specified as 30000; we now have 29996 words. Stopping reading.


## 1.3 加载预训练权重

In [16]:
embedding_matrix=load_embedding_matrix(max_vocab_size=params["vocab_size"])
embedding_matrix.shape

(30000, 300)

# 2. 模型训练

## 2.2 构建Encoder

In [ ]:
from seq2seq_tf2.seq2seq_model import Seq2Seq

In [ ]:
model=Seq2Seq(params,vocab)

# 2 读取训练好的模型

In [ ]:
from utils.config import checkpoint_dir,checkpoint_prefix

In [ ]:
ckpt = tf.train.Checkpoint(Seq2Seq=model)
ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_dir, max_to_keep=5)

In [21]:
ckpt.restore(ckpt_manager.latest_checkpoint)
if ckpt_manager.latest_checkpoint:
    print("Restored from {}".format(ckpt_manager.latest_checkpoint))
else:
    print("Initializing from scratch.")

Initializing from scratch.


# 3. 训练

In [ ]:
optimizer = tf.keras.optimizers.Adam(name='Adam',learning_rate=0.001)
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')


pad_index=vocab.PAD_TOKEN_INDEX

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, pad_index))
    loss_ = loss_object(real, pred)
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask
    return tf.reduce_mean(loss_)

In [ ]:
@tf.function
def train_step(dec_input, targ , enc_hidden):
    with tf.GradientTape() as tape:
      
        # 1. 构建encoder
        enc_output, enc_hidden = model.encoder(inp,enc_hidden)
        # 2. 复制
        dec_hidden = enc_hidden
        
        # 逐个预测序列 (dec_hidden, enc_output, dec_target)
        predictions, _ = model.teacher_decoder(dec_hidden, enc_output, targ)
        
        batch_loss = loss_function(targ, predictions)

        variables = model.trainable_variables
    
        gradients = tape.gradient(batch_loss, variables)

        optimizer.apply_gradients(zip(gradients, variables))

        return batch_loss

In [ ]:
epochs = 2 # params["epochs"]
# 如果检查点存在，则恢复最新的检查点。
if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print ('Latest checkpoint restored!!')
    
for epoch in range(epochs):
    start = time.time()
    enc_hidden = model.encoder.initialize_hidden_state()
    total_loss = 0

    for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
        batch_loss = train_step(inp, targ,enc_hidden)
        total_loss += batch_loss

        if batch % 20 == 0:
            print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                         batch,
                                                         batch_loss.numpy()))
    # saving (checkpoint) the model every 2 epochs
    if (epoch + 1) % 1 == 0:
        ckpt_save_path = ckpt_manager.save()
        print ('Saving checkpoint for epoch {} at {}'.format(epoch+1,
                                                             ckpt_save_path))

    print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                      total_loss / steps_per_epoch))
    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Latest checkpoint restored!!
Epoch 1 Batch 0 Loss 1.5986
Epoch 1 Batch 20 Loss 1.9914
Epoch 1 Batch 40 Loss 1.6394
Epoch 1 Batch 60 Loss 1.7797
Epoch 1 Batch 80 Loss 1.9847
Epoch 1 Batch 100 Loss 1.5832
Epoch 1 Batch 120 Loss 1.5847
Epoch 1 Batch 140 Loss 1.6309
Epoch 1 Batch 160 Loss 1.7777
Epoch 1 Batch 180 Loss 1.4987
Epoch 1 Batch 200 Loss 1.7672
Epoch 1 Batch 220 Loss 1.7807
Epoch 1 Batch 240 Loss 1.9074
Epoch 1 Batch 260 Loss 1.7454
Epoch 1 Batch 280 Loss 1.4510
Epoch 1 Batch 300 Loss 2.0057
Epoch 1 Batch 320 Loss 1.5629
Epoch 1 Batch 340 Loss 1.5472
Epoch 1 Batch 360 Loss 1.5364
Epoch 1 Batch 380 Loss 1.5812
Epoch 1 Batch 400 Loss 1.6733
Epoch 1 Batch 420 Loss 1.6366
Epoch 1 Batch 440 Loss 1.9129
Epoch 1 Batch 460 Loss 1.7522
Epoch 1 Batch 480 Loss 1.7613
Epoch 1 Batch 500 Loss 1.6329
Epoch 1 Batch 520 Loss 1.7082
Epoch 1 Batch 540 Loss 1.6982
Epoch 1 Batch 560 Loss 1.8837
Epoch 1 Batch 580 Loss 1.5187
Epoch 1 Batch 600 Loss 1.6717
Epoch 1 Batch 620 Loss 1.5868
Epoch 1 Batch 640

Time taken for 1 epoch 524.4936063289642 sec

# 载入模型

In [ ]:
ckpt.restore(ckpt_manager.latest_checkpoint)
if ckpt_manager.latest_checkpoint:
    print("Restored from {}".format(ckpt_manager.latest_checkpoint))
else:
    print("Initializing from scratch.")

Restored from /content/drive/My Drive/kaikeba/project01/code/data/checkpoints/training_checkpoints_seq2seq/ckpt-2


# 预测

In [ ]:
def evaluate(model,inputs,params):
    print('inputs:{}'.format(inputs))
    attention_plot = np.zeros((params['max_enc_len'],params['max_dec_len'],))
    
    inputs = tf.convert_to_tensor(inputs)

    pred_result = ''
    hidden = [tf.zeros((1, params['enc_units']))]
    enc_output, enc_hidden = model.encoder(inputs, hidden)

    dec_hidden = enc_hidden
    
    dec_input = tf.expand_dims([vocab.START_DECODING_INDEX], 0)

    for t in range(params['max_dec_len']):
        predictions, dec_hidden, attention_weights = model.decoder(dec_input,
                                                                  dec_hidden,
                                                                  enc_output)
        # storing the attention weights to plot later on
        attention_weights = tf.reshape(attention_weights, (-1, ))
        attention_plot[t] = attention_weights.numpy()
        predicted_id = tf.argmax(predictions[0]).numpy()
        pred_result += vocab.id2word[predicted_id] + ' '
        if predicted_id == vocab.STOP_DECODING_INDEX:
            return pred_result, attention_plot

        # the predicted ID is fed back into the model
        dec_input = tf.expand_dims([predicted_id], 0)

    return pred_result, attention_plot

In [ ]:
def translate(sentence):
    
    print('Input: %s' % (sentence))
    sentence_index = preprocess_sentence(sentence,params['max_dec_len'],vocab.word2id)
    pred_result, attention_plot = evaluate(model,sentence_index,params)

    print('sentence_index: %s' % (sentence_index))
    print('Predicted translation: {}'.format(pred_result))
    read_result=[vocab.id2word[index] for index in sentence_index.tolist()[0]]
    attention_plot = attention_plot[:len(pred_result.split(' ')), :len(read_result)]
    plot_attention(attention_plot, read_result, pred_result.split(' '))

## Restore the latest checkpoint and test

In [ ]:
sentence='哈佛H6换了正时皮带后报失火故障码是什么问题,技师说：你好！这种情况下我们就需要检验一下正时是否正确，可以通过测量发动机缸压来判断正时|车主说：正时确认对齐了|技师说：这款发动机是不是有平衡轴的？平衡轴也是需要对记号的,检查正时，测量缸压'

In [ ]:
translate(sentence)

# 批量预测

In [ ]:
def batch_predict(inps):
    # 判断输入长度
    batch_size=len(inps)
    # 开辟结果存储list
    preidicts=[''] * batch_size
    
    inps = tf.convert_to_tensor(inps)
    # 0. 初始化隐藏层输入
    hidden = [tf.zeros((batch_size, units))]
    # 1. 构建encoder
    enc_output, enc_hidden = model.encoder(inps, hidden)
    # 2. 复制
    dec_hidden = enc_hidden
    # 3. <START> * BATCH_SIZE 
    dec_input = tf.expand_dims([vocab['<START>']] * batch_size, 1)
    
    context_vector, _ = model.attention(dec_hidden, enc_output)
    # Teacher forcing - feeding the target as the next input
    for t in range(max_length_targ):
        # 计算上下文
        context_vector, attention_weights = model.attention(dec_hidden, enc_output)
        # 单步预测
        predictions, dec_hidden = model.decoder(dec_input,
                                         dec_hidden,
                                         enc_output,
                                         context_vector)
        
        # id转换 贪婪搜索
        predicted_ids = tf.argmax(predictions,axis=1).numpy()
        
        
        for index,predicted_id in enumerate(predicted_ids):
            preidicts[index]+= reverse_vocab[predicted_id] + ' '
        
        # using teacher forcing
        dec_input = tf.expand_dims(predicted_ids, 1)

    results=[]
    for preidict in preidicts:
        # 去掉句子前后空格
        preidict=preidict.strip()
        # 句子小于max len就结束了 截断
        if '<STOP>' in preidict:
            # 截断stop
            preidict=preidict[:preidict.index('<STOP>')]
        # 保存结果
        results.append(preidict)
    return results

In [ ]:
from tqdm import tqdm
import math

In [ ]:
def model_predict(data_X,batch_size):
    # 存储结果
    results=[]
    # 样本数量
    sample_size=len(data_X)
    # batch 操作轮数 math.ceil向上取整 小数 +1
    # 因为最后一个batch可能不足一个batch size 大小 ,但是依然需要计算  
    steps_epoch = math.ceil(sample_size/batch_size)
    # [0,steps_epoch)
    for i in tqdm(range(steps_epoch)):
        batch_data = data_X[i*batch_size:(i+1)*batch_size]
        results+=batch_predict(batch_data)
    return results

In [ ]:
%%time
results=model_predict(test_X,batch_size=64)


100%|██████████| 313/313 [01:50<00:00,  2.84it/s]

CPU times: user 1min 46s, sys: 6.67 s, total: 1min 53s
Wall time: 1min 50s


In [ ]:
results[1005]

'， 描述 ， 没有 问题 ， 需要 四 s 店 进行 清洗 '

In [ ]:
# 读入提交数据
test_df=pd.read_csv(test_data_path)
test_df.head()

,QID,Brand,Model,Question,Dialogue
0,Q1,大众(进口),高尔夫(进口),我的帕萨特烧机油怎么办怎么办？,技师说：你好，请问你的车跑了多少公里了，如果在保修期内，可以到当地的4店里面进行检查维修。如...
1,Q2,一汽-大众奥迪,奥迪A6,修一下多少钱是换还是修,技师说：你好师傅！抛光处理一下就好了！50元左右就好了，希望能够帮到你！祝你生活愉快！
2,Q3,上汽大众,帕萨特,帕萨特领域 喇叭坏了 店里说方向盘里线坏了 换一根两三百不等 感觉太贵,技师说：你好，气囊油丝坏了吗，这个价格不贵。可以更换。
3,Q4,南京菲亚特,派力奥,发动机漏气会有什么征兆？,技师说：你好！一：发动机没力，并伴有“啪啪”的漏气声音。二：发动机没力，并伴有排气管冒黑烟。...
4,Q5,东风本田,思铂睿,请问 那天右后胎扎了订，补了胎后跑高速80多开始有点抖，110时速以上抖动明显，以为是未做动...,技师说：你好师傅！可能前轮平衡快脱落或者不平衡造成的！建议前轮做一下动平衡就好了！希望能够帮...


# 判断是否有空值

In [ ]:
for idx,result in enumerate(results):
    if result=='':print(idx)

In [ ]:
# 赋值结果
test_df['Prediction']=results
#　提取ID和预测结果两列
test_df=test_df[['QID','Prediction']]

In [ ]:
test_df.head()

,QID,Prediction
0,Q1,， 现在 行驶 一千公里 ， 保修期 内 ， 4s店 内 4s店 进行 维修 ， 检查 机油...
1,Q2,！ 描述 ， 没有 等到 回复 ， 需要 300 元 左右 ！
2,Q3,， 描述 情况 分析 事故 车 ， 价格 不 贵 。
3,Q4,分析 诊断 电脑 读取 发动机 缸 垫 。
4,Q5,， 描述 ， 这种 情况 ， 轮胎 问题 ， 轮胎 问题 ， 轮胎 问题 ， 轮胎 问题 ，...


# 结果处理

In [ ]:
# 判断是否有空值
# for predic in test_df['Prediction']:
#     if type(predic) != str:
#         print(predic)

In [ ]:
test_df['Prediction']=test_df['Prediction'].apply(submit_proc)

In [ ]:
test_df.head()

,QID,Prediction
0,Q1,现在行驶一千公里，保修期内，4s店内4s店进行维修，检查机油消耗过大，更换机油消耗过大，检查...
1,Q2,描述，没有等到回复，需要300元左右！
2,Q3,描述情况分析事故车，价格不贵。
3,Q4,分析诊断电脑读取发动机缸垫。
4,Q5,描述，这种情况，轮胎问题，轮胎问题，轮胎问题，轮胎问题，轮胎问题，轮胎问题，轮胎问题，轮胎问...


# 保存结果

In [ ]:
from utils.file_utils import get_result_filename

In [ ]:
# 获取结果存储路径
result_save_path = get_result_filename(params["batch_size"],params["epochs"] , params["max_enc_len"], params["embedding_dim"],commit='_4_1_submit_proc_add_masks_loss_seq2seq_code')

In [ ]:
# 保存结果.
test_df.to_csv(result_save_path,index=None,sep=',')

In [ ]:
result_save_path

'/home/roger/kaikeba/03_lecture/code/result/2019_12_08_22_30_23_batch_size_64_epochs_5_max_length_inp_200_embedding_dim_500_4_1_submit_proc_add_masks_loss_seq2seq_code.csv'

In [ ]:
# 读取结果
test_df=pd.read_csv(result_save_path)
# 查看格式
test_df.head(10)

,QID,Prediction
0,Q1,现在行驶一千公里，保修期内，4s店内4s店进行维修，检查机油消耗过大，更换机油消耗过大，检查...
1,Q2,描述，没有等到回复，需要300元左右！
2,Q3,描述情况分析事故车，价格不贵。
3,Q4,分析诊断电脑读取发动机缸垫。
4,Q5,描述，这种情况，轮胎问题，轮胎问题，轮胎问题，轮胎问题，轮胎问题，轮胎问题，轮胎问题，轮胎问...
5,Q6,描述，这种情况，描述，这种情况，描述，这种情况，描述，这种情况，描述，这种情况，描述，这种情...
6,Q7,添加，添加防冻液，添加防冻液，添加防冻液，添加防冻液，添加防冻液，添加防冻液，添加防冻液，添...
7,Q8,机油灯亮，需要使用5w30机油，需要检查机油灯亮，需要使用5w30机油，需要检查机油灯亮，需...
8,Q9,图片来看，轮胎磨损严重，这种情况，轮胎磨损严重，这种情况，轮胎磨损严重，这种情况，轮胎磨损严...
9,Q10,这种情况可能天气凉情况下，长时间停放时间长以后，时间长以后，时间长以后，时间长以后，时间长以...


提交须知
请在提交之前仔细阅读“提交须知”。

1. 自动评审
系统根据选手提交的结果自动评分，提供每天5次的评测与排名机会，实时更新排行榜并按照评测分数从高到低排序。若一天内多次提交结果，新结果版本将覆盖原版本。

2. 评分标准
评测用到的核心算法为ROUGE(Recall-Oriented Understudy for Gisting Evaluation)，详见Wikipedia；具体用到的指标为ROUGE_L，即：Longest Common Subsequence (LCS) based statistics，关于LCS问题，详见Wikipedia。

所有参与评审的模型必须使用飞桨PaddlePaddle。所有参赛个人可无限使用基于AI Studio平台提供的训练资源。

3. 特别注意
选手需确认输出结果的总行数为20001（含表头），且QID ≤ Q20000，否则成绩无效。

# ROUGE

# 角度1：QA 问题

# 角度2： 摘要问题

# 角度3： 阅读理解问题

> 如果看成是阅读理解问题， 那么就是从Conversation中找出能回答Problem的答案， 由于目前的阅读理解数据集的答案长度通常比较短（一般是几个单词），所以state of the art的作法是根据Problem，从Context中选择一段作为答案，模型只要输出答案的开始和结束位置即可。 但是这个任务的report有点长，常常出现几十个甚至上百个词， 而且report中的词好像并不完全是来自于Conversation。 Report中67.7%的词来自于Conversation.

## Next steps

* [Download a different dataset](http://www.manythings.org/anki/) to experiment with translations, for example, English to German, or English to French.
* Experiment with training on a larger dataset, or using more epochs
* [Neural Machine Translation (seq2seq) Tutorial](https://github.com/tensorflow/nmt)
